In [3]:
import gradio as gr
import datetime


class VirtualAssistant:


    def __init__(self):
        # what stays remembered during the chat
        self.name = None
        self.mood = None
        self.lastTopic = None

        # details for summaries
        self.turns = 0
        self.topicsSeen = set()   
        self.moodHistory = []

        # placeholder
        self.useRealGguf = False

    def extractName(self, originalMessage, msgLower):
        """
        Try to pull a name out of sentences like:
        'my name is ____' or 'call me _____'
        """
        trigger = None
        if "my name is " in msgLower:
            trigger = "my name is "
        elif "call me " in msgLower:
            trigger = "call me "

        if trigger is None:
            return None

        startIndex =msgLower.find(trigger) + len(trigger)
        after =originalMessage[startIndex:].strip()

        # stop at punctuation
        for seperate in [",", ".", "!", "?"]:
            if seperate in after:
                parts = after.split(seperate, 1)
                after = parts[0].strip()

        if after == "":
            return None

        # make first letter uppercase so name looks normal by default
        return after[0].upper() + after[1:]

    def rememberTopic(self, intent):
        #Record rough topics for summary
        mapping = {
            "project_info": "the virtual assistant project",
            "project_tech": "how the project is implemented",
            "exam": "exams and studying",
            "study": "exams and studying",
            "mood": "how you were feeling",
        }
        if intent in mapping:
            self.topicsSeen.add(mapping[intent])

    def personalizeName(self, reply):
    
       # If we know the user's name add it where appropriate
    
        if self.name is None:
            return reply

        nameLower = self.name.lower()
        replyLower = reply.lower()

        if nameLower in replyLower:
            return reply

        return self.name + ", " + reply

    def matchCasing(self, userMessage, reply):

     #Matches users style

        hasLetter = False
        allLower = True

        for char in userMessage:
            if char.isalpha():
                hasLetter = True
                if not char.islower():
                    allLower = False
                    break

        # if user didn't type letters no change
        if not hasLetter:
            return reply

        # user uses all lowercase change whole reply (including name)
        if allLower:
            return reply.lower()

        # otherwise leave reply as written
        return reply

    # decision tree

    def detectIntent(self, msgLower):
        #Classify user message into a simple 'intent' using if/elif

        # simple "tokenization" so each sentence is broken into list of words
        cleaned = msgLower
        for char in ["?", "!", ",", "."]:
            cleaned = cleaned.replace(char, " ")
        words = cleaned.split()

        # greetings
        if ("hello" in words or "hi" in words or
            "hey" in words or "yo" in words):
            return "greeting"

        if ("bye" in words or "goodbye" in words):
            return "goodbye"

        if "see" in words and "you" in words:
            return "goodbye"

        # name
        if "my" in words and "name" in words and "is" in words:
            return "name"
        if "call" in words and "me" in words:
            return "name"
        if "my name is " in msgLower or "call me " in msgLower:
            return "name"

        # mood
        if "feel" in words or "feeling" in words:
            return "mood"
        if (("i'm" in words or "im" in words or "i" in words) and
            ("sad" in words or "tired" in words or "stressed" in words or "anxious" in words)):
            return "mood"

        # project info
        if ("what" in words and "project" in words and "about" in words):
            return "project_info"
        if "virtual" in words and "assistant" in words and "project" in words:
            return "project_info"
        if "what is this project" in msgLower:
            return "project_info"

        #project technical details
        if ("how does this work" in msgLower or
            "how do you work" in msgLower or
            "how are you implemented" in msgLower or
            "rule based" in msgLower or
            "gguf" in msgLower):
            return "project_tech"

        #study
        if ("exam" in msgLower or "final" in msgLower or
            "test" in msgLower or "quiz" in msgLower):
            return "exam"

        if ("study" in msgLower or "studying" in msgLower or
            "homework" in msgLower or "hw" in msgLower):
            return "study"

        # thanks
        if "thank" in msgLower or "thanks" in msgLower or "thx" in msgLower:
            return "thanks"

        # help
        if msgLower.strip() == "help" or "/help" in msgLower:
            return "help"

        # small talk
        if ("how" in words and "are" in words and "you" in words) or \
           ("what's" in words and "up" in words) or \
           ("whats" in words and "up" in words):
            return "small_talk"

        # summarize
        if ("summarize" in msgLower or
            "summary" in msgLower or
            "recap" in msgLower or
            "tl;dr" in msgLower or
            "tldr" in msgLower or
            "what did we talk about" in msgLower):
            return "summary"


    # handlers for every different branch

    def handleGreeting(self):
        self.lastTopic = "greeting"
        if self.name is not None:
            return "Nice to see you again."
        else:
            return (
                "Hey, I'm your virtual assistant.\n"
                "You can tell me your name by saying 'my name is ...'."
            )

    def handleGoodbye(self):
        self.lastTopic = "goodbye"
        return "Okay, bye for now. Thanks for chatting!"

    def handleName(self, originalMessage, msgLower):
        self.lastTopic = "name"
        name = self.extractName(originalMessage, msgLower)
        if name is None:
            return "I tried to understand your name, but I'm not sure. Try 'my name is ...'."
        self.name = name
        return "Nice to meet you, " + name + ". How are you feeling today?"

    def handleMood(self, originalMessage, msgLower):
        self.lastTopic = "mood"
        self.mood = originalMessage
        self.moodHistory.append(originalMessage)

        feelingWords = {
        #asked chatgpt for short reassurances based on moods to really nail the "llm reassurance" vibe
            "anxious": "It sounds like you’re anxious. That’s really tough, but I’m glad you said it out loud.",
            "stressed": "You sound stressed. Maybe break things into smaller tasks so it feels less impossible.",
            "overwhelmed": "Feeling overwhelmed is common with school. One tiny step at a time still counts.",
            "tired": "You’re tired. Please don’t underestimate sleep. Even a short break can help.",
            "sad": "I’m sorry you’re feeling sad. You don’t have to fix everything at once.",
            "depressed": "I’m really sorry you feel that way. Talking to someone you trust in real life can help too.",
            "good": "Nice, I’m glad you’re feeling good.",
            "happy": "Happy is always nice to hear. Share a bit of that if you can.",
        }

        for word in feelingWords:
            if word in msgLower:
                return feelingWords[word] + " If you want, you can tell me what’s going on."

        return (
            "Thank you for telling me how you feel: \""
            + originalMessage
            + "\".\nI might not fully understand it, but I’m here to listen."
        )

    def handleProjectInfo(self):
        self.lastTopic = "project_info"
        return (
            "This is my Virtual Assistant final project.\n"
            "- It uses simple if/elif rules to recognize patterns in what you type.\n"
            "- It keeps a tiny bit of memory (your name, mood, last topic) during the chat, "
            "which can be accessed by asking for a summary.\n"
            "- If a message doesn’t match any rule, it can fall back to a GGUF model "
            "to handle more open-ended questions."
        )

    def handleProjectTech(self):
        self.lastTopic = "project_tech"
        return ( 
            "Technically it works like this:\n"
            "1) Your message is converted to lowercase, and then broken up into a list of words. \n"
            "2) I run it through a decision tree of if/elif checks to detect an intention from the list of words \n"
            "   (greeting, name, mood, exam, etc.).\n"
            "3) Each intent has its own handler method that generates a predetermined reply.\n"
            "4) If nothing matches, I call a GGUF fallback function.\n"
        )

    def handleExam(self):
        self.lastTopic = "exam"
        return (
            "Exams are rough. Simple plan:\n"
            "1) Write down 3 topics you really need to review.\n"
            "2) Do about 25 minutes focused on the first one, then 5 minutes break.\n"
            "3) Repeat for the others.\n"
            "If you tell me the topic, I can try a small pep talk."
        )

    def handleStudy(self):
        self.lastTopic = "study"
        return (
            "For studying, smaller chunks help a lot:\n"
            "- Turn 'study everything' into 'do 3 practice problems' or 'review 2 pages'.\n"
            "- Start with one small piece so your brain doesn’t immediately check out."
        )

    def handleThanks(self):
        self.lastTopic = "thanks"
        return "You’re welcome!"

    def handleHelp(self):
        self.lastTopic = "help"
        return (
            "Things you can try saying:\n"
            "- 'hi', 'hello' (greeting)\n"
            "- 'my name is ...' (so I can remember your name)\n"
            "- 'I feel anxious / tired / sad / happy' (mood)\n"
            "- 'how are you?' or 'what's up' (small talk)\n"
            "- 'what is this project about?' (project info)\n"
            "- 'how do you work?' (technical explanation)\n"
            "- 'I have an exam' or 'I need to study' (study/exam help)\n"
            "- 'summarize our chat' (short recap)\n"
            "If I don’t understand, I’ll use my GGUF fallback or just admit I don’t know."
        )

    def handleSmallTalk(self):
        self.lastTopic = "small_talk"
        if self.mood is not None:
            return (
                "I'm all good. Earlier you told me: \""
                + self.mood
                + "\".\nSo I’d guess you’ve got a lot going on, but you’re still trying."
            )
        else:
            return "I’m doing as good as any simple program could be. How are you doing?"

    def handleSummary(self):
        #Symmary based on stored values
        self.lastTopic = "summary"

        if self.turns == 0:
            return "We haven’t really talked yet, so there isn’t much to summarize."

        lines = []

        if self.name:
            lines.append("- You told me your name is " + self.name + ".")

        if self.moodHistory:
            if len(self.moodHistory) == 1:
                lines.append("- You shared how you feel: \"" + self.moodHistory[0] + "\".")
            else:
                lines.append("- You shared how you felt at different times:")
                for feeling in self.moodHistory:
                    lines.append("  - \"" + feeling + "\"")

        if len(self.topicsSeen) > 0:
            topicsText = ", ".join(sorted(self.topicsSeen))
            lines.append("- We talked about: " + topicsText + ".")

        lines.append("- Altogether, we exchanged about " + str(self.turns) + " messages.")

        return "Here’s a quick summary of our chat so far:\n" + "\n".join(lines)



    def ggufFallback(self, userMessage):
        """
        Fallback for when no rule matches.

        Right now this is just a fixed message as a placeholder until implemenetd.
        """
        if not self.useRealGguf:
            return (
                "I don't have a specific rule for that.\n"
                "In the full design this is where I would call a local large language model "
                "to generate a more flexible answer."
            )

        return "GGUF mode is marked as on, but real integration is not implemented in this version."


    def respond(self, message):
        #main method for the ui
        text = message.strip()

        if text == "":
            reply = "Say something and I’ll try to respond."
            reply = self.matchCasing(message, reply)
            return reply

        self.turns += 1

        msgLower = text.lower()
        intent = self.detectIntent(msgLower)

        if intent == "greeting":
            reply = self.handleGreeting()
        elif intent == "goodbye":
            reply = self.handleGoodbye()
        elif intent == "name":
            reply = self.handleName(text, msgLower)
        elif intent == "mood":
            reply = self.handleMood(text, msgLower)
        elif intent == "project_info":
            reply = self.handleProjectInfo()
        elif intent == "project_tech":
            reply = self.handleProjectTech()
        elif intent == "exam":
            reply = self.handleExam()
        elif intent == "study":
            reply = self.handleStudy()
        elif intent == "thanks":
            reply = self.handleThanks()
        elif intent == "help":
            reply = self.handleHelp()
        elif intent == "small_talk":
            reply = self.handleSmallTalk()
        elif intent == "summary":
            reply = self.handleSummary()
        else:
            reply = self.ggufFallback(text)

        self.rememberTopic(intent)

        # incorporate the user's name if we know it
        reply = self.personalizeName(reply)

        # roughly match the user's casing style
        reply = self.matchCasing(message, reply)

        return reply


# Gradio UI set up
assistant = VirtualAssistant()

def chatFunction(message, history):

    return assistant.respond(message)

demo = gr.ChatInterface(
    fn=chatFunction,
    type="messages", 
    title="Virtual Assistant (Final Project)",
    description=(
        "Rule-based chatbot using if/elif, class for state, "
        "a simple summary feature, and a GGUF fallback."
    ),
)

demo.launch()


* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.
